### Preparing and pickling data for repeated use

The DATADIR constant and CATEGORIES arrays are to be adjusted to point to the location of the training data and to subfolders that contain the training data of different classes


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random

DATADIR = "data/SIDS/images"
CATEGORIES = ["ok_partA", "warn_partB", "alarm_partC"]

training_data = []

IMG_SIZE = 100

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
create_training_data()
print(len(training_data))


random.shuffle(training_data)

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(len(training_data), IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

### Model Creation

Different combinations of the desnse layer, kernel size and number of convulutional layers are tried out

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
import tensorflow.keras.backend as K
import math
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
from tensorflow.keras.callbacks import TensorBoard
import time

%matplotlib inline
np.random.seed(1)

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers =  [0]#[3, 4, 8]
layer_sizes =  [32]#[32, 64, 128]
conv_layers = [3]#[1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                
            model.add(Dropout(0.5))
            model.add(Dense(3))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='sparse_categorical_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=20,
                      validation_split=0.1,
                      callbacks=[tensorboard])
            model.save(NAME)

### Testing the webcam

In [ ]:
import cv2
camera = cv2.VideoCapture(1)
for i in range(3):
    return_value, image = camera.read()
    cv2.imwrite('opencv'+str(i)+'.jpeg', image)
del(camera)

### Running the model
Contiounuly takes in images off the webcam, applies the model, and if the baby is turning, leverages the AWS SNS to sent a notification

In [ ]:
import cv2
import tensorflow as tf
import os
import numpy as np
import boto3

def imageName(imageNumber):
    return 'opencv'+str(imageNumber)+'.jpeg'

def takePictures(numberOfPictures):
    camera = cv2.VideoCapture(1)
    for i in range(numberOfPictures):
        return_value, image = camera.read()
        cv2.imwrite(imageName(i), image)
    del(camera)
    
def prepare(filepath):
    filedir='out/dir/'  # this filedirectory need to exist for the images to be written
    IMG_SIZE = 100
    img_array = cv2.imread(os.path.join(filedir,filepath), cv2.IMREAD_GRAYSCALE)
    img_array = img_array/255
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

def findCategory(prediction_inner):
    p=prediction_inner
    a=np.argmax(p)
    return a-1


def alarmAlert():
    print("*******************************************************")
    print("*******************************************************")
    #push notification initiated
    client = boto3.client("sns",aws_access_key_id="XXXXXXXXXXXXXXXXXXXXXX",  # replace with the aws access key
                          aws_secret_access_key="XXXXXXXXXXXXXXXXXXXX",   # replace with the aws secret access key
                          region_name="us-east-2")
    client.publish(TopicArn="arn:aws:sns:us-east-2:470609516118:SidsAlert", # replace with the aws SNS topic Arn
                   Subject="SIDS Alert",
                   Message="Baby is turning")


try:
    while True:
        model = tf.keras.models.load_model("2-conv-32-nodes-0-dense-1606724863")
        frame1 = 1
        frame2 = 1
        frame3 = 1
        takePictures(3)
        prediction1 = model.predict([prepare(imageName(0))])
        #print(prediction1)
        prediction2 = model.predict([prepare(imageName(1))])
        #print(prediction2)
        prediction3 = model.predict([prepare(imageName(2))])
        #print(prediction3)
        frame1=findCategory(prediction1[0])
        #print(frame1)
        frame2=findCategory(prediction2[0])
        #print(frame2)
        frame3=findCategory(prediction3[0])
        #print(frame3)
        if(frame1==1) and (frame2==1) and (frame3==1):
            print("OK")
        elif(frame1==0) and (frame2==0) and (frame3==0):
            alarmAlert()
            os.system('say "the baby has turned over"')
        elif(frame1==2) and (frame2==2) and (frame3==2):
            print("WARN")
            alarmAlert()
            os.system('say "the baby is turning"')
except KeyboardInterrupt:
    pass

### Generate Images 
Gnerates images (by Rotating, panning, shearing, zooming, horizontally flipping) existing images

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tqdm import tqdm
import numpy as np


datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.05,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest')

DATADIR = "data/images/alarm_partC"
OUT_DATADIR = "data/genImage/alarm_partC"
LABEL = "alarm"
# Alarm has 130 raw images from videos
for img_file in tqdm(os.listdir(DATADIR)):
    img = load_img(os.path.join(DATADIR,img_file))  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    
    i = 0
    for batch in datagen.flow(x, batch_size=1,save_to_dir=OUT_DATADIR, save_prefix=LABEL, save_format='jpeg'):
        i +=1
        if i > 3: 
            break

DATADIR = "data/images/ok_partA"
OUT_DATADIR = "data/genImage/ok_partA"
LABEL = "ok"
# Alarm has 166 raw images from videos
for img_file in tqdm(os.listdir(DATADIR)):
    img = load_img(os.path.join(DATADIR,img_file))  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    
    i = 0
    for batch in datagen.flow(x, batch_size=1,save_to_dir=OUT_DATADIR, save_prefix=LABEL, save_format='jpeg'):
        i +=1
        if i > 3: 
            break

            
DATADIR = "data/images/warn_partB"
OUT_DATADIR = "data/genImage/warn_partB"
LABEL = "warn"
# Alarm has 166 raw images from videos
for img_file in tqdm(os.listdir(DATADIR)):
    img = load_img(os.path.join(DATADIR,img_file))  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    
    i = 0
    for batch in datagen.flow(x, batch_size=1,save_to_dir=OUT_DATADIR, save_prefix=LABEL, save_format='jpeg'):
        i +=1
        if i > 3: 
            break